# ngrams

> Functionality for ngram analysis.

In [ ]:
#| default_exp ngrams

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from spacy.attrs import ORTH, LOWER # remove? - add ENT_TYPE, ENT_IOB
import math
from fastcore.basics import patch

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE, set_logger_state

In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'
path_to_congress_corpus = f'{save_path}us-congressional-speeches-subset-100k.corpus'

In [ ]:
# #| hide
# for path in [path_to_toy_corpus, path_to_brown_corpus, path_to_reuters_corpus, path_to_congress_corpus]:
#     corpus = Corpus().load(path)
#     df = corpus.spaces.collect()
#     df = df.with_columns(
#         pl.col('position').shift(-1).alias('position2')
#     )
#     display(df.filter(pl.col('position') + 1 == pl.col('position2')).head())

In [ ]:
#| export
class Ngrams:
	""" Class for n-gram analysis reporting. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)
ngrams_toy = Ngrams(toy)

In [ ]:
#| exporti
@patch
def _get_ngrams(self:Ngrams, 
			   token_sequence: list[np.ndarray], # token sequence to get index for 
			   index_id: int, # index to search (i.e. ORTH, LOWER)
			   token_positions: list[np.ndarray], # positions of token sequence, returned by get_token_positions 
			   ngram_length: int = 2, # length of ngram
			   ngram_token_position: str = 'LEFT', # specify if token sequence is on LEFT or RIGHT (support for ngrams with token in middle of sequence is in-development))
			   exclude_punctuation:bool=False, # exclude ngrams with tokens
			   exclude_spaces:bool=False # ignores space tokens
			   ) -> np.ndarray: # array of ngrams results

	""" Get ngram data for a token sequence. """
	
	start_time = time.time()
	sequence_len = len(token_sequence[0])
	variants_len = len(token_sequence)
	token_index_len = len(token_positions[0])

	if index_id == ORTH:
		index = 'orth_index'
	else:
		index = 'lower_index'

	if ngram_token_position == 'RIGHT':
		ngram_range = range(-1 * ngram_length + sequence_len, sequence_len)
		ngrams = self.corpus.get_tokens_in_context(token_positions = token_positions, index = index, context_length = ngram_length, position_offset = 0, position_offset_step = -1, exclude_punctuation = False, exclude_spaces=exclude_spaces, convert_eof = False)
		ngrams = ngrams[::-1, :] # reversing order as retrieved right to left
	# elif ngram_token_position == 'MIDDLE': # further development needed - in roadmap
	# 	ngram_range = range(-1 * ngram_length + sequence_len + 1, sequence_len + 1)
	else:
		ngram_range = range(0, ngram_length)
		ngrams = self.corpus.get_tokens_in_context(token_positions = token_positions, index = index, context_length = ngram_length, position_offset = 0, position_offset_step = 1, exclude_punctuation = False, exclude_spaces=exclude_spaces, convert_eof = False)

	# old retrieval method - aligning with collocates context retrieval
	# ngrams = []
	
	# for pos in ngram_range:
	# 	if variants_len == 1 and pos > -1 and pos < sequence_len:
	# 		ngrams.append(np.full(token_index_len, token_sequence[0][pos]))
	# 	else:
	# 		seq = token_positions[0] + pos
	# 		ngrams.append(self.corpus.get_tokens_by_index(index)[seq])

	# ngrams = np.stack(ngrams)
	# logger.info(f'Ngrams ({ngrams.shape}) retrieval time: {(time.time() - start_time):.5f} seconds')

	# getting positions to search for EOF_TOKEN and filter out ngrams crossing doc boundaries
	#positions = (np.array(ngram_range)[:, None] != np.arange(sequence_len)).all(axis=1)
	# ngrams = np.delete(ngrams, np.where(ngrams[positions] == self.corpus.EOF_TOKEN)[1], axis=1)
	ngrams = np.delete(ngrams, np.where(ngrams == self.corpus.EOF_TOKEN)[1], axis=1)

	if exclude_punctuation: # see above - ngrams returned with punctuation tokens - and then cleaned if exclude_punctuation is True
		punctuation_tokens = self.corpus.punct_tokens
		ngrams = np.delete(ngrams, np.where(np.isin(ngrams, punctuation_tokens))[1], axis=1)

	logger.info(f'Ngrams ({ngrams.shape[1]}) retrieval time: {(time.time() - start_time):.5f} seconds')
	return ngrams


In [ ]:
#| hide
token_str = 'dog'
token_sequence, index_id = toy.tokenize(token_str, simple_indexing=True)
token_positions = toy.get_token_positions(token_sequence, index_id)
ngrams = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'LEFT')
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['dog', 'dog', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['sat', 'is', 'is'])

In [ ]:
#| hide
# set_logger_state('verbose')
# brown = Corpus().load(path_to_brown_corpus)
# ngrams_brown = Ngrams(brown)
# token_str = 'The'
# token_sequence, index_id = brown.tokenize(token_str, simple_indexing=True)
# token_positions = brown.get_token_positions(token_sequence, index_id)
# ngrams = ngrams_brown._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 3, ngram_token_position = 'RIGHT', exclude_punctuation = False, exclude_spaces = False)
# print(ngrams.shape)
# # find any ngrams[0] that contain brown.space_tokens
# from_x = 0
# to_x = 100
# df = pl.DataFrame([brown.token_ids_to_tokens(ngrams[0]),
#     brown.token_ids_to_tokens(ngrams[1]), brown.token_ids_to_tokens(ngrams[2])])
# display(df.with_row_index('row').filter((pl.col('column_1') == '\n') & 
#     (pl.col('row') >= from_x) & (pl.col('row') <= to_x)
# ).head(20))

# ngrams = ngrams_brown._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'RIGHT', exclude_punctuation = False, exclude_spaces = True)
# print(ngrams.shape)
# from_x = 0
# to_x = 100
# df = pl.DataFrame([brown.token_ids_to_tokens(ngrams[0]),
#     brown.token_ids_to_tokens(ngrams[1])])
# display(df.with_row_index('row').filter( 
#     (pl.col('row') == 37) | (pl.col('row') == 78) | (pl.col('row') == 94)).head(20))
# set_logger_state('quiet')

In [ ]:
# load the corpus
reuters = Corpus().load(path_to_reuters_corpus)

In [ ]:
# instantiate the Ngrams class
ngrams_reuters = Ngrams(reuters)

In [ ]:
#| hide
token_str = 'ocean'
token_sequence, index_id = reuters.tokenize(token_str, simple_indexing=True)
token_positions = reuters.get_token_positions(token_sequence, index_id)
ngrams = ngrams_reuters._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'LEFT')
print(ngrams)

[[48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119
  48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119
  48119 48119 48119 48119 48119]
 [13571 52710 46667 60529 46667 60529   936 71188 32924 72663 32924 72663
   9490 43372 26836 12053 53521 11720 13571 11720 11720 52211 13571 40736
  10743 67894 12439 65912 75228]]


In [ ]:
#| export
@patch
def ngrams(self: Ngrams, 
		   token_str: str, # token string to get ngrams for 
		   ngram_length:int = 2, # length of ngram
		   ngram_token_position: str = 'LEFT', # specify if token sequence is on LEFT or RIGHT (support for ngrams with token in middle of sequence is in-development))
		   normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
		   page_size:int = PAGE_SIZE, # number of results to display per results page 
		   page_current:int = 1, # current page of results
		   show_all_columns:bool = False, # return raw df with all columns or just ngram and frequency
		   exclude_punctuation:bool=True, # do not return ngrams with punctuation tokens
		   exclude_spaces:bool=True, # better thought of as ignore space tokens, unlikely that you will need this as anything other than True
		   use_cache:bool = True # retrieve the results from cache if available
		   ) -> Result: # return a Result object with ngram data
	""" Report ngram frequencies containing a token string. """

	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	start_time = time.time()
	cache_id = tuple(['ngram'] + list(token_sequence) + [ngram_length, ngram_token_position])

	if use_cache == True and cache_id in self.corpus.results_cache:
		logger.info('Using cached ngrams results')
		ngrams_report = self.corpus.results_cache[cache_id][0]
		total_unique = self.corpus.results_cache[cache_id][1]
		total_count = self.corpus.results_cache[cache_id][2]
	else:
		token_positions = self.corpus.get_token_positions(token_sequence, index_id)
		
		if len(token_positions[0]) == 0:
			logger.info('No tokens found')
			return Result(type = 'ngrams', df=pl.DataFrame(), title=f'Ngrams for "{token_str}"', description=f'No matches', summary_data={}, formatted_data=[])

		logger.info('Generating ngrams results')
		ngrams = self._get_ngrams(token_sequence, index_id, token_positions, ngram_length = ngram_length, ngram_token_position = ngram_token_position, exclude_punctuation=exclude_punctuation, exclude_spaces=exclude_spaces)
		total_count = ngrams.shape[1]
		schema = [f'token_{i+1}' for i in range(ngram_length)]
		ngrams_report = pl.DataFrame(ngrams.T, schema=schema).to_struct(name = 'ngram_token_ids').value_counts(sort=True).rename({"count": "frequency"})
		ngrams_report = ngrams_report.with_row_index(name='rank', offset=1)
		total_unique = len(ngrams_report)
		self.corpus.results_cache[cache_id] = (ngrams_report, total_unique, total_count)
	
	count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation, exclude_spaces)

	resultset_start = page_size*(page_current-1)

	# get specific chunk of report into polars based on resultset_start:
	ngrams_report_page = ngrams_report.slice(resultset_start, page_size).unnest('ngram_token_ids')
	ngrams_report_page = ngrams_report_page.with_columns(((pl.col("frequency") / pl.lit(count_tokens)) * normalize_by).alias('normalized_frequency'))

	token_strs = []
	for i in range(ngram_length):
		token_strs.append(self.corpus.token_ids_to_tokens(ngrams_report_page[f'token_{i+1}'].to_numpy()))
	token_strs = np.array(token_strs)
	ngram_text = [' '.join(column) for column in token_strs.T]
	ngrams_report_page = ngrams_report_page.with_columns(pl.Series(name="ngram", values=ngram_text))
	#ngrams_report_page = ngrams_report_page.to_pandas().set_index('rank')
	total_pages = math.ceil(total_unique/page_size)
	summary_data = {'ngram_length': ngram_length, 'ngram_token_position': ngram_token_position, 'total_unique': total_unique, 'total_count': total_count, 'page_current': page_current, 'total_pages': total_pages}
	formatted_data = [f'Report based on {tokens_descriptor}', f'Ngram length: {ngram_length}, Token position: {ngram_token_position.lower()}']

	if normalize_by is not None:
		formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	formatted_data.extend([f'Total unique ngrams: {total_unique:,}', f'Total ngrams: {total_count:,}'])

	if page_size != 0 and total_count > page_size:
		formatted_data.extend([f'Showing {min(page_size, total_count)} rows', f'Page {page_current} of {total_pages}']) 

	if show_all_columns == False:
		ngrams_report_page = ngrams_report_page[['rank', 'ngram', 'frequency', 'normalized_frequency']]
	
	logger.info(f'Ngrams report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'ngrams', df=ngrams_report_page, title=f'Ngrams for "{token_str}"', description=f'{self.corpus.name}', summary_data=summary_data, formatted_data=formatted_data)


In [ ]:
# run the ngrams method and display the results
ngrams_reuters.ngrams('environmental', ngram_length = 2, ngram_token_position = 'LEFT').display()

Ngrams for "environmental" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 environmental protection 
 4 
 0.03 
 
 
 2 
 environmental systems 
 4 
 0.03 
 
 
 3 
 environmental services 
 3 
 0.02 
 
 
 4 
 environmental damage 
 2 
 0.01 
 
 
 5 
 environmental regulations 
 2 
 0.01 
 
 
 6 
 environmental impact 
 2 
 0.01 
 
 
 7 
 environmental controls 
 1 
 0.01 
 
 
 8 
 environmental approval 
 1 
 0.01 
 
 
 9 
 environmental and 
 1 
 0.01 
 
 
 10 
 environmental sciences 
 1 
 0.01 
 
 
 11 
 environmental service 
 1 
 0.01 
 
 
 12 
 environmental concerns 
 1 
 0.01 
 
 
 13 
 environmental issues 
 1 
 0.01 
 
 
 14 
 environmental power 
 1 
 0.01 
 
 
 15 
 environmental plan 
 1 
 0.01 
 
 
 16 
 environmental had 
 1 
 0.01 
 
 
 17 
 environmental control 
 1 
 0.01 
 
 
 18 
 environmental management 
 1 
 0.01 
 
 
 19 
 environmental subsidiary 
 1 
 0.01 
 
 
 20 
 environmental was 
 1 
 0.01 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2, Token position: left 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 21 
 


 
 Total ngrams: 32 
 


 
 Showing 20 rows 
 


 
 Page 1 of 2

In [ ]:
# run the ngrams method and display the results
ngrams_reuters.ngrams('the highest', ngram_length = 3, ngram_token_position = 'LEFT', page_size = 10).display()

Ngrams for "the highest" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the highest since 
 7 
 0.05 
 
 
 2 
 the highest in 
 3 
 0.02 
 
 
 3 
 the highest level 
 2 
 0.01 
 
 
 4 
 the highest interest 
 2 
 0.01 
 
 
 5 
 the highest priority 
 2 
 0.01 
 
 
 6 
 the highest such 
 2 
 0.01 
 
 
 7 
 the highest positive 
 2 
 0.01 
 
 
 8 
 the highest ever 
 1 
 0.01 
 
 
 9 
 the highest depreciation 
 1 
 0.01 
 
 
 10 
 the highest rises 
 1 
 0.01 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3, Token position: left 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 28 
 


 
 Total ngrams: 41 
 


 
 Showing 10 rows 
 


 
 Page 1 of 3

In [ ]:
#| export
@patch
def ngram_frequencies(self: Ngrams, 
                ngram_length:int=2, # length of ngram
                case_sensitive:bool=False, # frequencies for tokens lowercased or with case preserved
				normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
				page_size:int=PAGE_SIZE, # number of rows to return
				page_current:int=1, # current page
				exclude_punctuation:bool=True, # exclude ngrams containing punctuation tokens
				exclude_spaces:bool=True # better thought of as ignore space tokens, unlikely that you will need this as anything other than True
				) -> Result: # return a Result object with the frequency table
	""" Report frequent ngrams. """
	
	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	start_time = time.time()

	if case_sensitive:
		index = 'orth_index'
	else:
		index = 'lower_index'

	filter = [self.corpus.EOF_TOKEN]
	if exclude_punctuation == True:
		filter += self.corpus.punct_tokens
	# if exclude_spaces == True: # changing this to ignore spaces
	# 	filter += self.corpus.space_tokens

	resultset_start = page_size*(page_current-1)

	count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation, exclude_spaces)
	formatted_data = [f'Report based on {tokens_descriptor}']
	formatted_data.append(f'Ngram length: {ngram_length}')

	# ngrams = self.corpus.get_ngrams_by_index(ngram_length = ngram_length, index = index).T

	ngrams_report = self.corpus.tokens.select(pl.col(index).alias('token_1')).with_row_index('position')
	if exclude_spaces: # using self.corpus.spaces - which has positions of space tokens - add new column is_space
		ngrams_report = ngrams_report.join(self.corpus.spaces.select(pl.col('position').alias('space_position'), pl.lit(True).alias('is_space')), left_on='position', right_on='space_position', how='left', maintain_order='left')
		ngrams_report = ngrams_report.filter(pl.col('is_space').is_null())

	ngrams_report = ngrams_report.with_columns([pl.col('token_1').shift(-i).alias(f'token_{i+1}') for i in range(1, ngram_length)])

	schema = [f'token_{i+1}' for i in range(ngram_length)]

	# ngrams_report = pl.LazyFrame(ngrams.T, schema=schema)
	ngrams_report = ngrams_report.group_by(schema).agg(pl.len().alias("frequency")).sort(by="frequency", descending=True)

	for i in range(ngram_length):
		ngrams_report = ngrams_report.filter(~pl.col(f'token_{i+1}').is_in(filter))

	total_unique = ngrams_report.select(pl.len()).collect().item()
	total_count = ngrams_report.select(pl.col('frequency').sum()).collect().item()

	ngrams_report_page = ngrams_report.slice(resultset_start, page_size).collect(engine = 'streaming')
	logger.info(f'collected report page: {(time.time() - start_time):.5f} seconds')
	token_strs = []
	for i in range(ngram_length):
		token_strs.append(self.corpus.token_ids_to_tokens(ngrams_report_page.select(pl.col(f'token_{i+1}')).to_numpy().flatten()))
	token_strs = np.array(token_strs)
	ngram_text = [' '.join(column) for column in token_strs.T]
	ngrams_report_page = ngrams_report_page.with_columns(pl.Series(name="ngram", values=ngram_text)).with_row_index(name='rank', offset=(page_current-1)*page_size+1)
	ngrams_report_page = ngrams_report_page.with_columns(((pl.col("frequency") / pl.lit(count_tokens)) * normalize_by).alias('normalized_frequency'))
	formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	
	formatted_data.extend([f'Total unique ngrams: {total_unique:,}', f'Total ngrams: {total_count:,}'])

	total_pages = math.ceil(total_unique/page_size)
	if page_size != 0 and total_count > page_size:
		formatted_data.extend([f'Showing {min(page_size, total_count)} rows', f'Page {page_current} of {total_pages}']) 

	ngrams_report_page = ngrams_report_page[['rank', 'ngram', 'frequency', 'normalized_frequency']]

	return Result(type = 'ngram_frequencies', df=ngrams_report_page, title=f'Ngram Frequencies', description=f'{self.corpus.name}', summary_data = {}, formatted_data = formatted_data)

In [ ]:
#| hide
# ngrams_report = brown.tokens.select(pl.col('lower_index').alias('token_1')).with_row_index('position').collect()
# ngrams_report = ngrams_report.join(brown.spaces.collect().select(pl.col('position').alias('space_position'), pl.lit(True).alias('is_space')), left_on='position', right_on='space_position', how='left', maintain_order='left')
# print(ngrams_report.filter(pl.col('position')>= 100).head(10))
# ngrams_report = ngrams_report.filter(pl.col('is_space').is_null())
# print(ngrams_report.shape)
# print(ngrams_report.filter(pl.col('position')>= 99).head(10))

In [ ]:
#| hide
set_logger_state('verbose')
ngrams_toy.ngram_frequencies(ngram_length = 2, case_sensitive = False, exclude_punctuation = True, page_current = 1).display()
ngrams_toy.ngrams('the').display()
set_logger_state('quiet')

2025-06-08 20:23:25 - INFO - ngram_frequencies - collected report page: 0.02983 seconds


Ngram Frequencies 
 
 
 Toy Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the dog 
 3 
 937.50 
 
 
 2 
 the cat 
 3 
 937.50 
 
 
 3 
 sat on 
 2 
 625.00 
 
 
 4 
 on the 
 2 
 625.00 
 
 
 5 
 dog is 
 2 
 625.00 
 
 
 6 
 the mat 
 2 
 625.00 
 
 
 7 
 cat is 
 2 
 625.00 
 
 
 8 
 cat sat 
 1 
 312.50 
 
 
 9 
 a tree 
 1 
 312.50 
 
 
 10 
 a hole 
 1 
 312.50 
 
 
 11 
 is barking 
 1 
 312.50 
 
 
 12 
 digging a 
 1 
 312.50 
 
 
 13 
 climbing a 
 1 
 312.50 
 
 
 14 
 is meowing 
 1 
 312.50 
 
 
 15 
 is digging 
 1 
 312.50 
 
 
 16 
 is climbing 
 1 
 312.50 
 
 
 17 
 dog sat 
 1 
 312.50 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 17 
 


 
 Total ngrams: 26 
 


 
 Showing 20 rows 
 


 
 Page 1 of 1

2025-06-08 20:23:25 - INFO - tokenize - Tokenization time: 0.00015 seconds
2025-06-08 20:23:25 - INFO - get_token_positions - Token indexing (8) time: 0.00004 seconds
2025-06-08 20:23:25 - INFO - ngrams - Generating ngrams results
2025-06-08 20:23:25 - INFO - get_tokens_in_context - Context tokens collected: (2, 8)
2025-06-08 20:23:25 - INFO - get_tokens_in_context - Context retrieved in 0.00 seconds.
2025-06-08 20:23:25 - INFO - _get_ngrams - Ngrams (8) retrieval time: 0.00128 seconds
2025-06-08 20:23:25 - INFO - ngrams - Ngrams report time: 0.00312 seconds


Ngrams for "the" 
 
 
 Toy Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the cat 
 3 
 937.50 
 
 
 2 
 the dog 
 3 
 937.50 
 
 
 3 
 the mat 
 2 
 625.00 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2, Token position: left 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 3 
 


 
 Total ngrams: 8

In [ ]:
#| hide
ngrams_reuters.ngrams('the company', ngram_length = 3).display()

Ngrams for "the company" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the company said 
 1,130 
 8.08 
 
 
 2 
 the company 's 
 447 
 3.20 
 
 
 3 
 the company also 
 106 
 0.76 
 
 
 4 
 the company has 
 65 
 0.46 
 
 
 5 
 the company to 
 61 
 0.44 
 
 
 6 
 the company reported 
 50 
 0.36 
 
 
 7 
 the company is 
 50 
 0.36 
 
 
 8 
 the company will 
 46 
 0.33 
 
 
 9 
 the company had 
 45 
 0.32 
 
 
 10 
 the company was 
 41 
 0.29 
 
 
 11 
 the company would 
 38 
 0.27 
 
 
 12 
 the company and 
 28 
 0.20 
 
 
 13 
 the company earned 
 21 
 0.15 
 
 
 14 
 the company did 
 20 
 0.14 
 
 
 15 
 the company today 
 15 
 0.11 
 
 
 16 
 the company for 
 14 
 0.10 
 
 
 17 
 the company in 
 13 
 0.09 
 
 
 18 
 the company added 
 13 
 0.09 
 
 
 19 
 the company last 
 11 
 0.08 
 
 
 20 
 the company might 
 10 
 0.07 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3, Token position: left 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 215 
 


 
 Total ngrams: 2,642 
 


 
 Showing 20 rows 
 


 
 Page 1 of 11

In [ ]:
ngrams_reuters.ngram_frequencies(ngram_length = 3, case_sensitive = False).display()

Ngram Frequencies 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the company said 
 474 
 3.39 
 
 
 2 
 mln dlrs in 
 325 
 2.32 
 
 
 3 
 cts vs loss 
 276 
 1.97 
 
 
 4 
 said it has 
 255 
 1.82 
 
 
 5 
 dlrs a share 
 245 
 1.75 
 
 
 6 
 pct of the 
 244 
 1.74 
 
 
 7 
 the end of 
 235 
 1.68 
 
 
 8 
 the company 's 
 224 
 1.60 
 
 
 9 
 the united states 
 202 
 1.44 
 
 
 10 
 cts net loss 
 199 
 1.42 
 
 
 11 
 mln avg shrs 
 198 
 1.42 
 
 
 12 
 inc said it 
 196 
 1.40 
 
 
 13 
 cts a share 
 196 
 1.40 
 
 
 14 
 the bank of 
 184 
 1.32 
 
 
 15 
 is expected to 
 181 
 1.29 
 
 
 16 
 cts prior pay 
 166 
 1.19 
 
 
 17 
 said in a 
 164 
 1.17 
 
 
 18 
 said it will 
 163 
 1.17 
 
 
 19 
 it said the 
 160 
 1.14 
 
 
 20 
 of the company 
 159 
 1.14 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 716,194 
 


 
 Total ngrams: 1,135,590 
 


 
 Showing 20 rows 
 


 
 Page 1 of 35810

In [ ]:
ngrams_reuters.ngram_frequencies(ngram_length = 3, case_sensitive = True).display()

Ngram Frequencies 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 The company said 
 384 
 2.75 
 
 
 2 
 mln dlrs in 
 356 
 2.55 
 
 
 3 
 cts vs loss 
 341 
 2.44 
 
 
 4 
 said it has 
 301 
 2.15 
 
 
 5 
 pct of the 
 293 
 2.09 
 
 
 6 
 mln Avg shrs 
 285 
 2.04 
 
 
 7 
 dlrs a share 
 279 
 1.99 
 
 
 8 
 cts Net loss 
 273 
 1.95 
 
 
 9 
 the end of 
 272 
 1.94 
 
 
 10 
 cts a share 
 271 
 1.94 
 
 
 11 
 the United States 
 262 
 1.87 
 
 
 12 
 the company 's 
 252 
 1.80 
 
 
 13 
 Inc said it 
 245 
 1.75 
 
 
 14 
 the company said 
 238 
 1.70 
 
 
 15 
 is expected to 
 227 
 1.62 
 
 
 16 
 cts vs profit 
 199 
 1.42 
 
 
 17 
 Corp said it 
 194 
 1.39 
 
 
 18 
 cts Oper net 
 184 
 1.32 
 
 
 19 
 cts prior Pay 
 183 
 1.31 
 
 
 20 
 said it will 
 183 
 1.31 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 733,074 
 


 
 Total ngrams: 1,135,590 
 


 
 Showing 20 rows 
 


 
 Page 1 of 36654

In [ ]:
#| hide
congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-100k.corpus')
report_congress = Ngrams(congress)
congress._init_token_arrays()

In [ ]:
#| hide
%time report_congress.ngram_frequencies(ngram_length = 2, case_sensitive = False, page_current = 1).display()

Ngram Frequencies 
 
 
 US Congressional Speeches Subset 100k 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 of the 
 129,042 
 71.61 
 
 
 2 
 in the 
 66,225 
 36.75 
 
 
 3 
 to the 
 65,139 
 36.15 
 
 
 4 
 the the 
 52,451 
 29.11 
 
 
 5 
 and the 
 36,773 
 20.41 
 
 
 6 
 that the 
 36,369 
 20.18 
 
 
 7 
 for the 
 28,266 
 15.69 
 
 
 8 
 the of 
 26,460 
 14.68 
 
 
 9 
 on the 
 25,200 
 13.98 
 
 
 10 
 it is 
 24,755 
 13.74 
 
 
 11 
 the to 
 22,957 
 12.74 
 
 
 12 
 by the 
 22,025 
 12.22 
 
 
 13 
 the senator 
 19,133 
 10.62 
 
 
 14 
 to be 
 17,872 
 9.92 
 
 
 15 
 the and 
 17,590 
 9.76 
 
 
 16 
 is the 
 17,304 
 9.60 
 
 
 17 
 with the 
 16,446 
 9.13 
 
 
 18 
 of this 
 15,750 
 8.74 
 
 
 19 
 the gentleman 
 14,720 
 8.17 
 
 
 20 
 senator from 
 14,306 
 7.94 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 2,050,099 
 


 
 Total ngrams: 16,133,676 
 


 
 Showing 20 rows 
 


 
 Page 1 of 102505

CPU times: user 31.9 s, sys: 854 ms, total: 32.8 s
Wall time: 2.35 s


In [ ]:
#| hide
%time report_congress.ngrams('senator', ngram_length = 2, ngram_token_position = 'RIGHT', page_size = 10).display()

Ngrams for "senator" 
 
 
 US Congressional Speeches Subset 100k 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the senator 
 37,269 
 20.68 
 
 
 2 
 distinguished senator 
 2,430 
 1.35 
 
 
 3 
 senior senator 
 1,099 
 0.61 
 
 
 4 
 junior senator 
 899 
 0.50 
 
 
 5 
 a senator 
 578 
 0.32 
 
 
 6 
 of senator 
 483 
 0.27 
 
 
 7 
 and senator 
 474 
 0.26 
 
 
 8 
 by senator 
 452 
 0.25 
 
 
 9 
 that senator 
 366 
 0.20 
 
 
 10 
 able senator 
 351 
 0.19 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2, Token position: right 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 774 
 


 
 Total ngrams: 49,510 
 


 
 Showing 10 rows 
 


 
 Page 1 of 78

CPU times: user 290 ms, sys: 80 ms, total: 370 ms
Wall time: 272 ms


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()